In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
from keras import layers
import string
import re
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/news.csv')
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


from matplotlib import pyplot as plt
df['Unnamed: 0'].plot(kind='hist', bins=20, title='Unnamed: 0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Unnamed: 0'].plot(kind='line', figsize=(8, 4), title='Unnamed: 0')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['label'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Unnamed: 0', y='label', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
df['label'] = df['label'].replace('REAL','0')
df['label'] = df['label'].replace('FAKE','1')
df['label'] = df['label'].astype('int32')

In [ ]:
df.drop(columns = ['title'], inplace=True)

In [ ]:
df

,Unnamed: 0,text,label
0,8476,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,It's primary day in New York and front-runners...,0
...,...,...,...
6330,4490,The State Department told the Republican Natio...,0
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,1
6332,8622,Anti-Trump Protesters Are Tools of the Oligar...,1
6333,4021,"ADDIS ABABA, Ethiopia —President Obama convene...",0


from matplotlib import pyplot as plt
df['Unnamed: 0'].plot(kind='hist', bins=20, title='Unnamed: 0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['label'].plot(kind='hist', bins=20, title='label')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='Unnamed: 0', y='label', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Unnamed: 0'].plot(kind='line', figsize=(8, 4), title='Unnamed: 0')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['label'].plot(kind='line', figsize=(8, 4), title='label')
plt.gca().spines[['top', 'right']].set_visible(False)

**Data Preparation**

In [ ]:
train_df, temp_df =  train_test_split(df, test_size=0.3, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, "<br />","")
  return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]","")

max_features = 20000
embedding_dim = 128
sequence_length = 500

**Text Vectorization**

In [ ]:
vectorize_layer = keras.layers.TextVectorization(standardize= custom_standardization, max_tokens=max_features, output_mode="int", output_sequence_length= sequence_length,)

In [ ]:
text_ds = tf.data.Dataset.from_tensor_slices(train_df['text'].values)
vectorize_layer.adapt(text_ds)

In [ ]:
def vectorize_text(text,label):
  text = tf.expand_dims(text, -1)
  label = tf.expand_dims(label, -1)
  return vectorize_layer(text), label
train_ds = tf.data.Dataset.from_tensor_slices((train_df['text'].values, train_df['label'].values))
train_ds = train_ds.map(vectorize_text)

val_ds = tf.data.Dataset.from_tensor_slices((valid_df['text'].values, valid_df['label'].values))
val_ds = val_ds.map(vectorize_text)

test_ds = tf.data.Dataset.from_tensor_slices((test_df['text'].values,test_df['label'].values))
test_ds = test_ds.map(vectorize_text)

train_ds = train_ds.cache().prefetch(buffer_size = 10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size = 10)

**Training Models**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epochs = 5
model.fit(train_ds,validation_data=val_ds, epochs=epochs, batch_size=64)

Epoch 1/5
4434/4434 [==============================] - 204s 46ms/step - loss: 0.3798 - accuracy: 0.8223 - val_loss: 0.2620 - val_accuracy: 0.8947
Epoch 2/5
4434/4434 [==============================] - 203s 46ms/step - loss: 0.1273 - accuracy: 0.9533 - val_loss: 0.3354 - val_accuracy: 0.9053
Epoch 3/5
4434/4434 [==============================] - 199s 45ms/step - loss: 0.0610 - accuracy: 0.9802 - val_loss: 0.4951 - val_accuracy: 0.9105
Epoch 4/5
4434/4434 [==============================] - 199s 45ms/step - loss: 0.0389 - accuracy: 0.9880 - val_loss: 0.5989 - val_accuracy: 0.9242
Epoch 5/5
4434/4434 [==============================] - 198s 45ms/step - loss: 0.0426 - accuracy: 0.9917 - val_loss: 0.7498 - val_accuracy: 0.9232
